In [ ]:
#### FIRST ATTEMPT TO DO THE GEOJSON

import geopandas as gpd

# Check the georeference stystem
gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Chile/ColonelFires/Progression_ColonelFire_UTC.shp')
print(gdf.crs)

# If the result is EPSG:32719 -> Coordenate System iks WGS84 / UTM zone 19S -> NO REPROJECTION NEEDED!!!

EPSG:32719


In [ ]:
# EDIT SHAPEFILE WITH ISOCHRONES #

gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Chile/ColonelFires/Progression_ColonelFire_UTC.shp')

# Rename attributes
# Chance 'old_name':'new_name'
gdf = gdf.rename(columns={
    'date_hour': 'iso_timestamp'
})

# Add new attributes
gdf['area_ha'] = gdf.geometry.area / 10000  
gdf['perimeter_m'] = gdf.geometry.length

# Sort ids with time stamps -> from early to later
gdf = gdf.sort_values('iso_timestamp')

# Convine geometries with the same 'iso_timestamp'
gdf_2 = gdf.dissolve(by='iso_timestamp', as_index=False)

# CReate a new 'id' secuancial (starting in 1)
gdf_2 = gdf_2.sort_values('iso_timestamp').reset_index(drop=True)
gdf_2['id'] = gdf_2.index + 1

# Define the attribute colums order
mand_atrib= ['id', 'iso_timestamp', 'area_ha', 'perimeter_m']

# Add the rest of the attributes
extra_atrib = [col for col in gdf_2.columns if col not in mand_atrib and col != 'geometry']

# Reorganizar columnas (la geometría al final)
gdf_2 = gdf_2[mand_atrib + extra_atrib]

# Ahora guarda el archivo con las columnas en el orden deseado
gdf_2.to_file('ColonelFires_test5.geojson', driver='GeoJSON')
